<a href="https://colab.research.google.com/github/OMAR200002/NLP/blob/main/TP3_Khadrouni_Omar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Charger le Dataset

In [ ]:
df = pd.read_csv('movie_review.csv')

texts = df['text']
tags = df['tag']

# Pré-traitement

In [ ]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word not in stop_words and word not in string.punctuation]
    return filtered_words

preprocessed_texts = [preprocess_text(text) for text in texts]
print(preprocessed_texts[0])

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', "'re", 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', "'s", 'never', 'really', 'comic', 'book', 'like', 'hell']


# Entraînement du Modèle Word2Vec


In [ ]:
word2vec_model = Word2Vec(preprocessed_texts, vector_size=100, window=5, min_count=1, workers=4)

# Vectorisation des Textes

In [ ]:
def vectorize_text(text, model, size=100):
    vectors = [model.wv[word] for word in text if word in model.wv]

    if len(vectors) > 0:
        vector = np.mean(vectors, axis=0)
    else:
        # Si aucun mot n'est reconnu, renvoyer un vecteur de zéros
        vector = np.zeros(size)

    return vector

vectorized_texts = [vectorize_text(text, word2vec_model) for text in preprocessed_texts]

# Convertir en array numpy pour éviter les problèmes de dimensionnalité
vectorized_texts = np.array(vectorized_texts)
print(vectorized_texts)

[[-0.4068065   0.71108294  0.41838363 ... -0.55216956 -0.02672342
   0.1244285 ]
 [-0.16390195  0.53877532  0.33321279 ... -0.26485941 -0.0094007
   0.29539573]
 [-0.28882986  0.7675522   0.40230328 ... -0.60826433 -0.10781863
   0.20013359]
 ...
 [-0.69750112  0.97282124  0.76180422 ... -0.6776737  -0.42151785
   0.3792586 ]
 [-0.16064388  0.48637897  0.4668203  ... -0.42024243 -0.04850768
   0.22736402]
 [-0.59965068  0.84847343  0.43310422 ... -0.6950165  -0.34589958
  -0.03605396]]


# Division des Données

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectorized_texts, tags, test_size=0.2)

# Construction et Évaluation du modéle

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         neg       0.56      0.47      0.51      6373
         pos       0.56      0.65      0.60      6571

    accuracy                           0.56     12944
   macro avg       0.56      0.56      0.56     12944
weighted avg       0.56      0.56      0.56     12944

